In [12]:
!pip install numpy
!pip install boto3
import numpy as np
import utils
import boto3
import pickle

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [13]:
import os

BASE_IMAGE = 'docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime'
bucket_name = 'elyra'
S3_END_POINT = "http://minio-kubeflow.apps.cluster-3f07.3f07.sandbox333.opentlc.com"
S3_ACCESS_ID = "minio"
S3_ACCESS_KEY = "minio123"

In [14]:
s3_endpoint_url = 'http://minio-kubeflow.apps.cluster-3f07.3f07.sandbox333.opentlc.com'
s3_access_key = "minio"
s3_secret_key = "minio123"

s3 = boto3.client(service_name='s3',
              	aws_access_key_id = s3_access_key,
              	aws_secret_access_key = s3_secret_key,
              	endpoint_url=s3_endpoint_url)

s3.list_buckets()['Buckets']

[{'Name': 'elyra',
  'CreationDate': datetime.datetime(2021, 5, 11, 16, 15, 17, 5000, tzinfo=tzlocal())},
 {'Name': 'mlpipeline',
  'CreationDate': datetime.datetime(2021, 5, 11, 13, 48, 5, 478000, tzinfo=tzlocal())}]

In [15]:
def write_to_store(bucket, data, key):
    s3.put_object(Body=pickle.dumps(data),
                      Bucket=bucket,
                      Key=key)
    
def gen_hyperparam_grid() -> int:
    '''Generate a list of namedtuples
    of hyperparams to evaluate
    '''

    grid = []
    for num_hidden_layers in [1,2,3]:
        for num_nodes in [1,2,3]:
            for activation in ['relu', 'sigmoid']:
                grid.append({'num_hidden_layers': num_hidden_layers,
                             'num_nodes': num_nodes,
                             'activation': activation
                            })


    write_to_store(bucket_name, grid, 'hyperparam_grid')

    return len(grid)

In [16]:
retcode = gen_hyperparam_grid()
if retcode <= 0: raise ValueError("Step 1b failed")